<a href="https://colab.research.google.com/github/Ramane23/peft_fine_tuning/blob/main/peft_fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Step 1: Install Required Libraries
!pip install transformers datasets evaluate peft torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
## Step 2: Import Basic Libraries
import numpy as np
import pandas as pd
import torch

In [3]:
## Step 3: Import HuggingFace Libraries
from datasets import load_dataset
import evaluate

# Import libraries for working with pre-trained models
from transformers import (
    AutoModelForSequenceClassification,  # For loading models for classification
    AutoTokenizer,                       # For tokenizing text input
    TrainingArguments,                   # For configuring the training process
    Trainer                              # For handling the training loop
)

# Import PEFT-specific libraries
from peft import (
    LoraConfig,                          # For configuring LoRA
    get_peft_model,                      # For applying PEFT to a model
    TaskType,                            # For specifying the task type
    PeftModel                            # For loading saved PEFT models
)

In [4]:
## Step 4: Load a Dataset
# Load the emotion dataset
# This dataset contains text samples labeled with emotions
dataset = load_dataset("emotion")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
# Display basic information about the dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [6]:
# Show the first few examples
print("\n--- Examples from the dataset ---")
for i in range(3):
    print(f"Example {i+1}:")
    print(f"Text: {dataset['train'][i]['text']}")
    print(f"Label: {dataset['train'][i]['label']}")
    print()


--- Examples from the dataset ---
Example 1:
Text: i didnt feel humiliated
Label: 0

Example 2:
Text: i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake
Label: 0

Example 3:
Text: im grabbing a minute to post i feel greedy wrong
Label: 3



In [10]:
## Step 5: Explore the Dataset
# Get information about the labels
label_names = dataset['train'].features['label'].names
num_labels = len(label_names)

# Create mappings between label IDs and names
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in enumerate(label_names)}

print(f"the list of labels is {label_names}")
print(f"Number of labels: {num_labels}")
print(f"Labels: {id2label}")

the list of labels is ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
Number of labels: 6
Labels: {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}


In [11]:
# Get dataset sizes
print(f"\nTrain set size: {len(dataset['train'])}")
print(f"Validation set size: {len(dataset['validation'])}")
print(f"Test set size: {len(dataset['test'])}")


Train set size: 16000
Validation set size: 2000
Test set size: 2000


In [12]:
## Step 6: Load a Pre-trained Model and Tokenizer
# Load the pre-trained GPT-2 model configured for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=num_labels,        # Number of classes for classification
    id2label=id2label,            # Mapping from ID to label name
    label2id=label2id,            # Mapping from label name to ID
)

# Load the GPT-2 tokenizer
# The tokenizer converts text to token IDs that the model can understand
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# GPT-2 doesn't have a padding token by default, so we set it to the EOS token
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]